In [1]:
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import calendar
import bandit.main as bd

In [2]:
currDate = date.today()
currDay = calendar.day_name[currDate.weekday()]
currDay

'Tuesday'

In [3]:
daynames = [calendar.day_name[x] for x in range(7)]
weekdays = pd.DataFrame({'Weekday': daynames})
weekdays['Date'] = currDate

In [4]:
goals = pd.DataFrame({'Goal': range(1,22)})
goals['Date'] = currDate

In [5]:
goal_schedule = weekdays.merge(goals, how='left', on = 'Date')
goal_schedule['Complete'] = np.nan
#days_and_stores.drop('key',1, inplace=True)

In [6]:
sheet_info = df = pd.read_excel(io='DailyGoal.xlsx')

In [7]:
subprocess.Popen('DailyGoal.xlsx', shell=True)

In [8]:
goal_schedule = goal_schedule.append(sheet_info)

In [9]:
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [10]:
def df_bandit_diff(df):
    dfgb = df.groupby(['Goal'])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [11]:
goal_schedule['Cohort'] = (goal_schedule['Goal'] - 1) // 7
goal_schedule['Class'] = goal_schedule['Weekday'] + '-' + goal_schedule['Goal'].map(str)
df = goal_schedule.reset_index(drop=True)

In [12]:
df['Score'] = np.where(df['Complete'] >= df['Goal'], 1, 0)
df.loc[df['Complete'].isnull(), 'Score'] = np.NaN
df = bd.reduce(df,'Class')

4 0.99 4


In [13]:
df_quality = bd.get_quality(df['Score'])
df = df.join(df_quality)
df.tail(5)

,Weekday,Date,Goal,Complete,Cohort,Class,Score,Quality
146,Sunday,2018-09-11,21,NaN,2,Sunday-21,NaN,NaN
147,Friday,2018-09-07 00:00:00,1,21.0,0,Friday-1,1.0,NaN
148,Saturday,2018-09-08 00:00:00,1,13.0,0,Saturday-1,1.0,NaN
149,Sunday,2018-09-09 00:00:00,1,11.0,0,Sunday-1,1.0,NaN
150,Monday,2018-09-10 00:00:00,1,NaN,0,Monday-1,NaN,NaN


In [14]:
# pick the current day
df = df[df['Weekday'] == currDay]

In [15]:
df_class = df_bandit_class(df,'Cohort')
df_class

,Cohort,count,mean,var,total,donext
0,0,0,NaN,NaN,0,999.0
1,1,0,NaN,NaN,0,999.0
2,2,0,NaN,NaN,0,999.0


In [16]:
nextClass = df_class.iloc[0,0]

In [17]:
if df['Complete'].isnull().sum() > 0:
    df_filtered = df[df['Cohort'] == nextClass]
else:
    df_filtered = df

In [18]:
currDay

'Tuesday'

In [19]:
df_bandit_diff(df_filtered)

,Goal,count,mean,var,total,donext
0,1,0,NaN,NaN,0,999.0
1,2,0,NaN,NaN,0,999.0
2,3,0,NaN,NaN,0,999.0
3,4,0,NaN,NaN,0,999.0
4,5,0,NaN,NaN,0,999.0
5,6,0,NaN,NaN,0,999.0
6,7,0,NaN,NaN,0,999.0
